In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [2]:
import urllib.request  # lib that handles URLs
import io
import pandas as pd
pd.options.display.max_rows=250
import numpy as np
from datetime import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter

import pyspark.sql.functions as func

In [93]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline, Model
from pyspark.sql.functions import col

In [75]:
df_events = spark.read.parquet("s3://bigdataproject-pr/filtered_events.parquet")
df_events.cache()

DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: int, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]

In [76]:
continuousCols = ["GoldsteinScale", "NumMentions", "NumSources", "NumArticles"]

for col in continuousCols:
    df_events = df_events.withColumn(col, func.col(col).cast("double"))

In [89]:
df_events = df_events.withColumn("MonthYear", func.col("MonthYear").cast("string"))
df_events_2017 = df_events.where(df_events.MonthYear.like("2017%"))
df_events_2018 = df_events.where(df_events.MonthYear.like("2018%"))


In [33]:
df_events.printSchema

<bound method DataFrame.printSchema of DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: date, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: double, NumSources: double, NumArticles: double, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]>

In [94]:
stringInputs = ["Actor1Name", "Actor2Name", "EventCode", "Actor1Geo_FullName", "Actor2Geo_FullName", 
                "ActionGeo_CountryCode","SOURCEURL"]


indexers = [ StringIndexer(inputCol=column, outputCol=column + "_IX", handleInvalid= "skip") 
            for column in stringInputs ]

encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers]

assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols, outputCol="features")



In [96]:

rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=10)

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [ ]:
pipeline = Pipeline(stages=[indexers, encoders, assembler, rf, labelConverter])